# Сравнение метрик обнаружения текстовых блоко YOLOv8 nano без/с предобработкой изображений

In [1]:
# Импорт библиотек
import cv2
import numpy as np
from pathlib import Path
from ultralytics import YOLO
from tabulate import tabulate
import yaml
import tempfile
import shutil
from tqdm import tqdm

In [ ]:
def calculate_iou(box1, box2):
    """Вычисление Intersection over Union для двух bounding box"""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    return inter_area / (box1_area + box2_area - inter_area)

def load_yolo_annotations(label_path, img_width, img_height):
    """Загрузка аннотаций в формате YOLO"""
    boxes = []
    if label_path.exists():
        with open(label_path) as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    x_center = float(parts[1]) * img_width
                    y_center = float(parts[2]) * img_height
                    width = float(parts[3]) * img_width
                    height = float(parts[4]) * img_height
                    boxes.append([
                        x_center - width/2,  # x1
                        y_center - height/2,  # y1
                        x_center + width/2,   # x2
                        y_center + height/2   # y2
                    ])
    return boxes

def preprocess_image(image_path):
    """Функция предобработки изображения"""
    img = cv2.imread(str(image_path))
    if img is None:
        raise ValueError(f"Не удалось загрузить изображение: {image_path}")

    # Пример предобработки: конвертация в grayscale и обратно в BGR
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    processed_img = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    return processed_img

def evaluate_detections(model, test_images_dir, test_labels_dir, preprocess=False):
    """Ручной расчет метрик Precision, Recall и F1-score"""
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    total_gt = 0
    total_images = 0

    for img_path in tqdm(list(test_images_dir.glob('*.jpg')), desc="Обработка изображений"):
        try:
            # Загрузка изображения
            if preprocess:
                img = preprocess_image(img_path)
            else:
                img = cv2.imread(str(img_path))
                if img is None:
                    raise ValueError(f"Не удалось загрузить изображение: {img_path}")

            h, w = img.shape[:2]

            # Загрузка ground truth
            label_path = test_labels_dir / f"{img_path.stem}.txt"
            gt_boxes = load_yolo_annotations(label_path, w, h)
            total_gt += len(gt_boxes)

            # Детекция
            results = model(img, conf=0.25, iou=0.45, verbose=False)[0]
            pred_boxes = results.boxes.xyxy.cpu().numpy()

            # Сопоставление предсказаний с GT
            matched_gt = set()
            for pred_box in pred_boxes:
                best_iou = 0
                best_idx = -1

                for i, gt_box in enumerate(gt_boxes):
                    if i in matched_gt:
                        continue

                    iou = calculate_iou(pred_box, gt_box)
                    if iou > best_iou:
                        best_iou = iou
                        best_idx = i

                if best_iou > 0.5:  # Порог IoU
                    true_positives += 1
                    matched_gt.add(best_idx)
                else:
                    false_positives += 1

            false_negatives += len(gt_boxes) - len(matched_gt)
            total_images += 1

        except Exception as e:
            print(f"Ошибка при обработке {img_path.name}: {str(e)}")
            continue

    # Расчет метрик
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return {
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'true_positives': true_positives,
        'false_positives': false_positives,
        'false_negatives': false_negatives,
        'total_gt': total_gt,
        'total_images': total_images
    }

def evaluate_map(model, data_yaml, test_images_dir, preprocess=False):
    """Вычисление mAP метрик с возможностью предобработки"""
    if not preprocess:
        # Без предобработки - используем стандартный val
        return model.val(
            data=data_yaml,
            split='test',
            imgsz=640,
            conf=0.25,
            iou=0.45
        )
    else:
        # С предобработкой - создаем временный датасет
        with tempfile.TemporaryDirectory() as temp_dir:
            temp_dir = Path(temp_dir)

            # Создаем структуру папок
            (temp_dir / 'images').mkdir(parents=True, exist_ok=True)
            (temp_dir / 'labels').mkdir(parents=True, exist_ok=True)

            # Копируем labels из оригинального датасета
            original_labels_dir = test_images_dir.parent / 'labels'
            for label_file in original_labels_dir.glob('*.txt'):
                shutil.copy(label_file, temp_dir / 'labels')

            # Обрабатываем и сохраняем изображения
            for img_file in tqdm(list(test_images_dir.glob('*.jpg')), desc="Предобработка изображений"):
                try:
                    processed_img = preprocess_image(img_file)
                    output_path = temp_dir / 'images' / img_file.name
                    cv2.imwrite(str(output_path), processed_img)
                except Exception as e:
                    print(f"Ошибка при обработке {img_file.name}: {str(e)}")
                    continue

            # Создаем временный data.yaml
            with open(data_yaml) as f:
                data = yaml.safe_load(f)

            # Обновляем пути и добавляем фиктивные train/val
            data['test'] = str(temp_dir)
            data['train'] = str(temp_dir)  # Фиктивный путь
            data['val'] = str(temp_dir)    # Фиктивный путь

            # Сохраняем количество классов и имена классов
            if 'nc' not in data:
                data['nc'] = len(data['names']) if 'names' in data else 1
            if 'names' not in data:
                data['names'] = ['object'] * data['nc']

            temp_data_yaml = temp_dir / 'data.yaml'
            with open(temp_data_yaml, 'w') as f:
                yaml.dump(data, f)

            # Вычисляем метрики
            return model.val(
                data=str(temp_data_yaml),
                split='test',  # Используем только test набор
                imgsz=640,
                conf=0.25,
                iou=0.45
            )


In [ ]:
# Основной код

# Пути к данным и модели
model_path = "/content/drive/MyDrive/VKR/yolov8n_custom2/detect/yolov8_custom/weights/best.pt"
data_yaml = '/content/drive/MyDrive/VKR/kpk_dataset-9/data.yaml'
test_images_dir = Path('/content/drive/MyDrive/VKR/kpk_dataset-9/test/images')
test_labels_dir = Path('/content/drive/MyDrive/VKR/kpk_dataset-9/test/labels')

# 1. Загрузка модели
model = YOLO(model_path)

# 2. Вычисление метрик без предобработки
print("Вычисление метрик БЕЗ предобработки...")
metrics_raw = evaluate_map(model, data_yaml, test_images_dir, preprocess=False)
manual_metrics_no_pp = evaluate_detections(model, test_images_dir, test_labels_dir, preprocess=False)

# 3. Вычисление метрик с предобработкой
print("\nВычисление метрик С предобработкой...")
metrics_preprocessed = evaluate_map(model, data_yaml, test_images_dir, preprocess=True)
manual_metrics_with_pp = evaluate_detections(model, test_images_dir, test_labels_dir, preprocess=True)

# 4. Формирование таблицы сравнения
comparison_data = [
    ["Метрика", "Без предобработки", "С предобработкой"],
    ["Обработано изображений", manual_metrics_no_pp['total_images'], manual_metrics_with_pp['total_images']],
    ["Всего текстовых блоков (GT)", manual_metrics_no_pp['total_gt'], manual_metrics_with_pp['total_gt']],
    ["Обнаружено блоков",
        manual_metrics_no_pp['true_positives'] + manual_metrics_no_pp['false_positives'],
        manual_metrics_with_pp['true_positives'] + manual_metrics_with_pp['false_positives']],
    ["True Positives", manual_metrics_no_pp['true_positives'], manual_metrics_with_pp['true_positives']],
    ["False Positives", manual_metrics_no_pp['false_positives'], manual_metrics_with_pp['false_positives']],
    ["False Negatives", manual_metrics_no_pp['false_negatives'], manual_metrics_with_pp['false_negatives']],
    ["Precision (P)", f"{manual_metrics_no_pp['precision']:.4f}", f"{manual_metrics_with_pp['precision']:.4f}"],
    ["Recall (R)", f"{manual_metrics_no_pp['recall']:.4f}", f"{manual_metrics_with_pp['recall']:.4f}"],
    ["F1-score", f"{manual_metrics_no_pp['f1_score']:.4f}", f"{manual_metrics_with_pp['f1_score']:.4f}"],
    ["mAP@0.5",
        f"{metrics_raw.results_dict.get('metrics/mAP50(B)', 0):.4f}",
        f"{metrics_preprocessed.results_dict.get('metrics/mAP50(B)', 0):.4f}"],
    ["mAP@0.5-0.95",
        f"{metrics_raw.results_dict.get('metrics/mAP50-95(B)', 0):.4f}",
        f"{metrics_preprocessed.results_dict.get('metrics/mAP50-95(B)', 0):.4f}"]
]

# 5. Вывод результатов
print("\nСравнение метрик детекции текстовых блоков:")
print(tabulate(comparison_data, headers="firstrow", tablefmt="grid"))